In [9]:
!pip install tqdm
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install bitsandbytes
!pip install qwen_vl_utils

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 50.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import re
import torch
import csv
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from datasets import load_dataset
from PIL import Image

In [12]:
from transformers import Qwen2_5_VLForConditionalGeneration
from transformers import AutoTokenizer, AutoProcessor

In [3]:
# 1. Setup model & tokenizer
# ---------------------------
model_path = "saves/Qwen2.5-VL-7B-Instruct-Lora-SFT-Checkpoint11000"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
    load_in_4bit = True,
    torch_dtype=torch.bfloat16,
).eval()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [4]:
# 2. Load your test image
image_path = "test.png"
image = Image.open(image_path).convert("RGB")  # ensure RGB format


In [13]:
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
# Load image
image = Image.open("test.png").convert("RGB")

# Build prompt
instruction = (
    "Given the following image of a skin flap, classify it into one of these categories:\n"
    "0 - Viable skin\n"
    "1 - Skin with Venous Problems\n"
    "2 - Skin with Arterial Problems\n"
    "3 - Temporary Hypoperfused skin, yet viable\n"
    "4 - Necrotic skin\n"
    "5 - Scarred skin\n\n"
    "Format your response as:\n"
    "Classification: X\n"
    "Explanation: <explanation here>"
)

# Build messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": instruction}
        ],
    }
]

# Process vision inputs
from qwen_vl_utils import process_vision_info
text_prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)

# Process full inputs
inputs = processor(
    text=[text_prompt],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to(model.device)

# Generate output
with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.1,
        top_p=0.1,
    )

# Decode output
generated_ids_trimmed = generated_ids[:, inputs.input_ids.shape[1]:]
output_text = processor.batch_decode(
    generated_ids_trimmed,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False,
)[0]

print("Model Output:\n", output_text)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Model Output:
 Classification: 0
Explanation: The image shows a person's face with a sticker-like overlay. There is no visible skin flap or any indication of skin issues such as venous problems, arterial problems, temporary hypoperfusion, necrosis, or scarring. The image does not provide enough context to classify the skin in this manner.
